In [3]:
import matplotlib.pyplot as plt
import os
import time
import sys

In [ ]:
!mo --saved_model_dir ./000000001/ --data_type FP32 --output_dir models/resnet_50/FP32 --input_shape [1,180,180,3]

In [ ]:
!mo --saved_model_dir ./000000001/ --data_type FP16 --output_dir models/resnet_50/FP16 --input_shape [1,180,180,3]

In [ ]:
%%writefile ov_classification_job.sh

OUTPUT_FILE=$1
DEVICE=$2
FP_MODEL=$3
INPUT_PATH=$4
NUM_REQS=5$5

echo VENV_PATH=$VENV_PATH
echo OPENVINO_RUNTIME=$OPENVINO_RUNTIME
echo INPUT_FILE=$INPUT_FILE
echo FP_MODEL=$FP_MODEL
echo INPUT_TILE=$INPUT_FILE
echo NUM_REQS=$NUM_REQS

# Follow this order of setting up environment for openVINO 2022.1.0.553
echo "Activating a Python virtual environment from ${VENV_PATH}..."
source ${VENV_PATH}/bin/activate
echo "Activating OpenVINO variables from ${OPENVINO_RUNTIME}..."
source ${OPENVINO_RUNTIME}/setupvars.sh

# The default path for the job is the user's home directory,
#  change directory to where the files are.
cd $PBS_O_WORKDIR

# Make sure that the output directory exists.
mkdir -p $OUTPUT_FILE

# Set inference model IR files using specified precision
MODELPATH=./models/resnet_50/${FP_MODEL}/saved_model.xml
LABEL_FILE=./model_classes.txt
INPUT_PATH=./inferece_images/


python3 classification_demo.py -i $INPUT_PATH -m $MODELPATH -o $OUTPUT_FILE --labels $LABEL_FILE -nireq $NUM_REQS



# python3 classification_demo.py -m <path_to_classification_model> \
#                                -i <path_to_folder_with_images> \
#                                 --labels <path_to_file_with_list_of_labels>

In [ ]:
%%writefile ov_classify_job.sh

INPUT=$1
FP_MODEL=$2
DEVICE=$3
OUTPUT=$4

# Follow this order of setting up environment for openVINO 2022.1.0.553
echo "Activating a Python virtual environment from ${VENV_PATH}..."
source ${VENV_PATH}/bin/activate
echo "Activating OpenVINO variables from ${OPENVINO_RUNTIME}..."
source ${OPENVINO_RUNTIME}/setupvars.sh

MODELPATH=./models/resnet_50/${FP_MODEL}/saved_model.xml

# The default path for the job is the user's home directory,
#  change directory to where the files are.
cd $PBS_O_WORKDIR

python3 ov-classify.py 

In [2]:
%%writefile tf_classify_job.sh

INPUT=$1
FP_MODEL=$2
DEVICE=$3
OUTPUT=$4

    

# Follow this order of setting up environment for openVINO 2022.1.0.553
echo "Activating a Python virtual environment from ${VENV_PATH}..."
source ${VENV_PATH}/bin/activate
echo "Activating OpenVINO variables from ${OPENVINO_RUNTIME}..."
source ${OPENVINO_RUNTIME}/setupvars.sh



# The default path for the job is the user's home directory,
#  change directory to where the files are.
cd $PBS_O_WORKDIR

python3 tf-classify.py

Overwriting tf_classify_job.sh


In [ ]:
!pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c

In [ ]:
# job_id_core = !qsub ov_classification_job.sh -l nodes=1:idc073 -F "./results/core/ CPU FP32 0.4" -N classification_core -v VENV_PATH,OPENVINO_RUNTIME
# print(job_id_core[0])

In [1]:
job_id_ov_core = !qsub ov_classify_job.sh -l nodes=1:idc073 -F "./results/core_ov/" -N classify_core_ov -v VENV_PATH,OPENVINO_RUNTIME
print(job_id_ov_core[0])

546405.v-qsvr-1.devcloud-edge


In [ ]:
if cpu:
    job_id_cpu_ov = !qsub ov_classify_job.sh -l nodes=1:idc073 -F "./results/core_ov/ CPU FP32 " -N classify_core_ov -v VENV_PATH,OPENVINO_RUNTIME
print(job_id_ov_core[0])
    job_id_cpu_tf = !qsub tf_classify_job.sh -l nodes=1:idc073 -F "./results/core_ov/ CPU FP32" -N classify_core_ov -v VENV_PATH,OPENVINO_RUNTIME
print(job_id_ov_core[0])
if gpu:
job_id_ov_fp16 = !qsub ov_classify_job.sh -l nodes=1:idc073 -F "./results/core_ov/ GPU FP16" -N classify_core_ov -v VENV_PATH,OPENVINO_RUNTIME
print(job_id_ov_core[0])
job_id_ov_fp32 = !qsub ov_classify_job.sh -l nodes=1:idc073 -F "./results/core_ov/ GPU FP32" -N classify_core_ov -v VENV_PATH,OPENVINO_RUNTIME
print(job_id_ov_core[0])

In [3]:
job_id_tf_core = !qsub tf_classify_job.sh -l nodes=1:idc073 -F "./results/core_tf/" -N classify_core_tf -v VENV_PATH,OPENVINO_RUNTIME
print(job_id_tf_core[0])

536229.v-qsvr-1.devcloud-edge


In [ ]:
from qarpo.demoutils import *
liveQstat()

In [4]:
job_id_ov = job_id_ov_core[0].split('.')[0]
job_id_ov = 'o'+job_id_ov
count = 0
while not os.path.exists('./classify_core_ov.{}'.format(job_id_ov)):  # Wait until the stats file is created.
    time.sleep(1)
    count = count + 1
    assert count != 180, "Job did not finish" #Wait to for 3 mins to check if job gets done, and break from the infinite loop
x_ov = open("./classify_core_ov.{}".format(job_id_ov), "r")
import re
stats = {}
for fps_ov in x_ov:
    if re.search('FPS', fps_ov):
        print(fps_ov)
        stats['FPS_OV'] = str(fps_ov.split(" ")[1])
    if re.search('Inference time', fps_ov):
        print(fps_ov)
        stats['Time_OV'] = str(fps_ov.split(" ")[2])
# print(stats)

FPS: 54.77 

Inference time: 18.26 ms



In [6]:
job_id_tf = job_id_tf_core[0].split('.')[0]
job_id_tf = 'o'+job_id_tf
count = 0
while not os.path.exists('./classify_core_tf.{}'.format(job_id_tf)):  # Wait until the stats file is created.
    time.sleep(1)
    count = count + 1
    assert count != 180, "Job did not finish" #Wait to for 3 mins to check if job gets done, and break from the infinite loop
x_tf = open("./classify_core_tf.{}".format(job_id_tf), "r")
import re
for fps_tf in x_tf:
    if re.search('FPS', fps_tf):
        print(fps_tf)
        stats['FPS_TF'] = str(fps_tf.split(" ")[1])
    if re.search('Inference time', fps_tf):
        print(fps_tf)
        stats['Time_TF'] = str(fps_tf.split(" ")[2])
# print(stats)

FPS: 32.12 fps



NameError: name 'stats' is not defined

In [ ]:
!accuracy_check -c resnet50-accuracy-check.yml -m ./models/resnet_50/FP16/ -s ./inference_images/ -a ./annotations/

In [ ]:
stats_fps = {key: stats[key] for key in stats.keys()
                               & {'FPS_OV', 'FPS_TF'}}
print(stats_fps)
stats_time = {key: stats[key] for key in stats.keys()
                               & {'Time_OV', 'Time_TF'}}
print(stats_time)

In [ ]:
import matplotlib.pyplot as plt
import json

x = ['Intel-core,i5-12500te']
xAxis_ov = [key for key, value in stats_ov.items()]
yAxis_ov = [value for key, value in stats_ov.items()]

fig = plt.figure()
plt.bar(xAxis_ov,yAxis_ov, color='blue')
plt.xlabel('variable')
plt.ylabel('value')